# Uploading and Using Custom Images in Amazon SageMaker

In the second part, the focus is on integrating the custom Docker image with a Amazon SageMaker Domain. This section will cover important aspects such as setting up SageMaker environments, specifying the right configurations, and ensuring seamless integration of the custom image with SageMaker's capabilities.

In [1]:
import boto3
import json
import sagemaker
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
sm = boto3.client("sagemaker")

To tailor the SageMaker environment to specific needs, create a configuration file named `app-image-config-input.json`
. This configuration is essential for setting up custom kernel environments in SageMaker's kernel gateway images.

**Key Aspects of the Configuration**
KernelSpecs Name Matching: The Name value under `KernelSpecs` *must exactly match the name* of the `kernelSpec` available in the Docker image associated with this AppImageConfig. This is case-sensitive. To find the available kernelSpecs in an image, run jupyter kernelspec list from a shell inside the container.

You can debug your image locally by running:

`docker run -it -p 8888:8888 {your_account_id}.dkr.ecr.{region}.amazonaws.com/repository_name:image_name bash`

MountPath for EFS: MountPath specifies the path within the image where the [Amazon Elastic File System (Amazon EFS)](https://aws.amazon.com/es/efs/) home directory will be mounted. This path should be distinct from any paths used within the container.

In [3]:
%%writefile app-image-config-input.json
{
    "AppImageConfigName": "test-image-config",
    "KernelGatewayImageConfig": {
        "KernelSpecs": [
            {
                "Name": "env1",
                "DisplayName": "Python 3.11 [pip env: Faker]"
            },
            {
                "Name": "env2",
                "DisplayName": "Python 3.11 [pip env: matplotlib]"
            }
        ],
        "FileSystemConfig": {
            "MountPath": "/root",
            "DefaultUid": 0,
            "DefaultGid": 0
        }
    }
}

Overwriting app-image-config-input.json


In [4]:
with open("app-image-config-input.json", "rb") as f:
    app_image_config = json.loads(f.read())

In [5]:
app_image_config

{'AppImageConfigName': 'test-image-config',
 'KernelGatewayImageConfig': {'KernelSpecs': [{'Name': 'env1',
    'DisplayName': 'Python 3.11 [pip env: Faker]'},
   {'Name': 'env2', 'DisplayName': 'Python 3.11 [pip env: matplotlib]'}],
  'FileSystemConfig': {'MountPath': '/root',
   'DefaultUid': 0,
   'DefaultGid': 0}}}

This section of the notebook focuses on extracting essential details from the SageMaker notebook's metadata file and using them to construct the full name of a Docker image located in Amazon ECR.

In [6]:
role = sagemaker.get_execution_role()

NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        md = json.loads(f.read())
        print(f"Notebook metadata: {md.keys()}")

Notebook metadata: dict_keys(['ResourceArn', 'ResourceName'])


In [7]:
repo_name = "sagemaker-custom-images"
image_name = "test-image"
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = md["ResourceArn"].split(":")[3]
full_name = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{repo_name}:{image_name}"

#print(full_name)

sm.delete_image(ImageName=image_name)

In [ ]:
# Create Studio Image
r = sm.create_image(
    Description="Test with custom environments",
    DisplayName="Test image",
    ImageName=image_name,
    RoleArn=role,
)

r

Now create a SageMaker image version from the container image.

In [ ]:
# Create image version
r = sm.create_image_version(
    BaseImage=full_name,
    ImageName=image_name,
)

r

In [10]:
r = sm.describe_image_version(ImageName=image_name)
print(f"Image version status: {r['ImageVersionStatus']}")

Image version status: CREATED


The image config is created based on the `app-image-config-input.json`. 

- This chunk will fail if there's already an image config with the same name. You can delete it with: `sm.delete_app_image_config(AppImageConfigName=app_image_config["AppImageConfigName"])`
- To retrieve the metadata of the image config, run `sm.describe_app_image_config(AppImageConfigName="test-image-config")`
- To list all image configs, run `sm.list_app_image_configs()`

In [ ]:
# create image config
r = sm.create_app_image_config(
    AppImageConfigName=app_image_config["AppImageConfigName"],
    KernelGatewayImageConfig=app_image_config["KernelGatewayImageConfig"],
)

r

Update an existing SageMaker domain to use this app image. If you don't have a domain, you must create a new one.



In [12]:
%%writefile update-domain-input.json
{
    "DefaultUserSettings": {
        "KernelGatewayAppSettings": {
            "CustomImages": [
                {
                    "ImageName": "test-image",
                    "AppImageConfigName": "test-image-config"
                }
            ]
        }
    }
}

Overwriting update-domain-input.json


In [13]:
with open("update-domain-input.json", "rb") as f:
    update_domain_input = json.loads(f.read())

Now you need to retrieve the `DomainId` of the specific domain where you intend to update the image. In this chunk, you accomplish this by listing all the domains that are currently in service and selecting one from the list.

In [14]:
domains = [d for d in sm.list_domains()["Domains"] if d["Status"] == 'InService']

domains[5]

{'DomainArn': 'arn:aws:sagemaker:us-east-1:035665124469:domain/d-8pjidge43uiu',
 'DomainId': 'd-8pjidge43uiu',
 'DomainName': 'soap-notes',
 'Status': 'InService',
 'CreationTime': datetime.datetime(2023, 5, 31, 8, 40, 13, 508000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 1, 15, 17, 15, 13, 456000, tzinfo=tzlocal()),
 'Url': 'https://d-8pjidge43uiu.studio.us-east-1.sagemaker.aws'}

In [15]:
domains[5]['DomainId']

'd-8pjidge43uiu'

To programmatically update the domain, the execution role associated with the notebook instance must have the necessary permissions. Consider the following policy as an example of the required rights:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sagemaker:UpdateDomain",
            "Resource": "arn:aws:sagemaker:{region}:{account_id}:domain/d-8pjidge43uiu"
        }
    ]
}
```

You can see the information of the domain by running `sm.describe_domain(DomainId="d-8pjidge43uiu")`

In [ ]:
r = sm.update_domain(
    DomainId=domains[5]['DomainId'],
    DefaultUserSettings=update_domain_input["DefaultUserSettings"],
)

r

It takes around 5 minutes for the image to be visibe within Sagemaker Studio. You can select your image and kernel like this:

![](img/select_image.png)

Now test both environments:

![](img/env_faker.png)

![](img/env_matplotlib.png)

## Update an image

You can use the `sm.create_image_version` to push a new version of your image.

```python
r = sm.create_image_version(
    BaseImage=full_name,
    ImageName=image_name,
)

r
```

## References

1. [Dive Deep into Amazon SageMaker Studio Notebook Architecture (AWS Blogs)](https://aws.amazon.com/blogs/machine-learning/dive-deep-into-amazon-sagemaker-studio-notebook-architecture/)
2. [Four Approaches to Manage Python Packages in Amazon SageMaker Studio Notebooks (AWS Blogs)](https://aws.amazon.com/blogs/machine-learning/four-approaches-to-manage-python-packages-in-amazon-sagemaker-studio-notebooks/)
3. [Create Custom App - Amazon SageMaker Studio Package Management (GitHub)](https://github.com/aws-samples/amazon-sagemaker-studio-package-management/blob/master/notebooks/create-custom-app.ipynb)
4. [Amazon SageMaker Studio Custom Image Samples (GitHub)](https://github.com/aws-samples/sagemaker-studio-custom-image-samples/tree/main)
5. [Kernelspecs - Jupyter Client Documentation](https://jupyter-client.readthedocs.io/en/latest/kernels.html#kernelspecs)
6. [Amazon SageMaker Studio Custom Image Specifications (AWS Documentation)](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-byoi-specs.html)